In [1]:
from qiskit import *
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
import itertools
import numpy as np
from time import sleep
import pandas as pd
import progressbar
import csv
sim = Aer.get_backend('qasm_simulator')
%matplotlib inline

c:\Users\taren\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\taren\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\taren\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Find all permutations and synonyms

In [2]:
gates = ['x', 'y', 'z', 'h']
gateCombos = []
comboSV = []
similar = []

In [3]:
#Find all permutations and append their statevectors and permutations to lists
for L in range(1, len(gates)+1):
    
    #for subset in itertools.permutations(gates, L):
    for subset in itertools.product(gates, repeat=L):
        #print(subset)
        gateCombos.append(subset)
#Add a no gate option
gateCombos.append('')
        
#find statevectors of each possible permutation and store in list
for i in range(len(gateCombos)):
    qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
    #print(gateCombos[i])
    
    for j in gateCombos[i]:
        getattr(qc , j)(0)
        #print(qc)
    # Let's get the result:
    qc.save_statevector()
    qobj = assemble(qc)
    result = sim.run(qobj).result()
    # Print the statevector neatly:
    final_state = result.get_statevector()
    array_to_latex(final_state, prefix="\\text{Statevector = }")
    comboSV.append(final_state)


In [4]:
len(gateCombos)
print(gateCombos[len(gateCombos)-1])

In [5]:
len(comboSV)
print(comboSV[len(comboSV)-1])

Statevector([1.+0.j, 0.+0.j],
            dims=(2,))


In [6]:
#Create a list of synonym pairs based on if statevectors are equal
bar = progressbar.ProgressBar(maxval=len(comboSV)).start()
for i in range(len(comboSV)):
    for j in range(len(comboSV)):
        temp = []
        if(i != j and comboSV[i]==comboSV[j]):
            temp.append(gateCombos[i])
            temp.append(gateCombos[j])
            #print(temp)
            similar.append(temp)
    bar.update(i)
            

In [7]:
len(similar)

8330

In [8]:
bar = progressbar.ProgressBar(maxval=len(similar)).start()
# provider = IBMQ.get_provider('ibm-q')
# backend = provider.get_backend('ibmq_armonk')
#sort largest synonym in pair first
#If synonyms are the same in length, check computing times of each and set slower one as target and quicker one as the replacement
listToRemove = []
for i in range(len(similar)):
    if len(similar[i][1]) > len(similar[i][0]):
        similar[i].insert(0, similar[i].pop(1))
    elif len(similar[i][1]) == len(similar[i][0]):
        #1
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for j in similar[i][0]:
            getattr(qc , j)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        time1 = result.time_taken
        #2
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for j in similar[i][1]:
            getattr(qc , j)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        time2 = result.time_taken
        if time2 > time1:
            similar[i].insert(0, similar[i].pop(1))
        if time2 == time1:
            listToRemove.append(i)
    bar.update(i)
listToRemove.sort(reverse=True)
print("Removing Elements at Indices: " + str(listToRemove))
listToRemove.sort(reverse=True)
for r in listToRemove:
    similar.pop(r)

Removing Elements at Indices: [8277, 8271, 8233, 8229, 8190, 8142, 8140, 8071, 7936, 7935, 7910, 7897, 7886, 7800, 7798, 7744, 7735, 7727, 7726, 7657, 7625, 7489, 7398, 7358, 7352, 7331, 7319, 7223, 7220, 7090, 7088, 7037, 7012, 6966, 6798, 6792, 6631, 6485, 6460, 6391, 6373, 6328, 6268, 6167, 6140, 6134, 6115, 6111, 5959, 5899, 5885, 5860, 5848, 5801, 5772, 5771, 5646, 5576, 5557, 5534, 5532, 5435, 5310, 5254, 5246, 5147, 4914, 4907, 4811, 4698, 4668, 4580, 4410, 4409, 4407, 4399, 4353, 4218, 4217, 4211, 4177, 4169, 4167, 4133, 4016, 3919, 3818, 3816, 3795, 3722, 3675, 3647, 3596, 3595, 3575, 3535, 3483, 3478, 3461, 3444, 3408, 3407, 3400, 3384, 3322, 3300, 3207, 3130, 3120, 3086, 2868, 2834, 2829, 2784, 2665, 2514, 2484, 2468, 2443, 2398, 2379, 2340, 2217, 2088, 1851, 1767, 1609, 1608, 1157, 1056, 1032, 1030, 1028, 898, 700]


In [9]:
len(similar)

8195

## Create Database File

In [11]:
with open(r"Synonym_Database\key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(similar)



### CSV to Pandas DF

In [12]:
df = pd.read_csv(r"Synonym_Database\key.csv")

### Filtering out multiple replacements for one target

In [13]:
unique = df['Target'].unique()
unique = unique.tolist()
targets = df['Target'].tolist()

In [14]:
bar = progressbar.ProgressBar(maxval=len(unique)).start()
remove = []
for i in range(len(unique)):
    bar.update(i)
    times = []
    indices = []
    for j in range(len(targets)):
        if unique[i] == targets[j]:
            indices.append(j)
    for x in range(len(indices)):
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for q in similar[x][1]:
            getattr(qc , q)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        times.append(result.time_taken)
    least = min(times)
    for e in indices:
        remove.append(e)
    remove.remove(indices[times.index(least)])
remove.sort(reverse=True)
#print(remove)
for i in remove:
    similar.pop(i)

In [15]:
len(similar)

328

In [16]:
with open(r"Synonym_Database\key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(similar)

## Reading and Cleaning File

In [17]:
#Get simplist replacements for each target
keyDF = pd.read_csv(r"Synonym_Database\key.csv")

for item in range(len(keyDF)):
    for m in range(0, 50):
        for element in range(len(keyDF)):
            if keyDF['Target'][element] == keyDF['Replacement'][item]:
                # print("1. " + str(keyDF['Replacement'][item]))
                # print("2. " + str(keyDF['Target'][element]))
                keyDF['Replacement'][item] = keyDF['Replacement'][element]

lst = []
print(keyDF)
for i in range(len(keyDF)):
    temp = []
    temp.append(keyDF['Target'][i])
    temp.append(keyDF['Replacement'][i])
    lst.append(temp)
with open(r"Synonym_Database\key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(lst)

                   Target Replacement
0         ('h', 'z', 'h')      ('x',)
1              ('z', 'y')      ('y',)
2         ('y', 'y', 'y')      ('y',)
3    ('h', 'h', 'z', 'y')      ('y',)
4              ('z', 'h')      ('h',)
..                    ...         ...
323  ('h', 'h', 'h', 'h')         NaN
324                ('z',)         NaN
325            ('z', 'z')         NaN
326            ('h', 'h')         NaN
327  ('y', 'y', 'y', 'y')         NaN

[328 rows x 2 columns]
